In [15]:
from google.colab import drive, files
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
from keras import applications
from keras import optimizers
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, InputLayer
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

In [0]:
DATA_AUGMENTATION = False
DATASET_NAME = 'miniMIT_Etus'

ROOT_PATH    = 'drive/My Drive/medical_image_recognition/'
DATASET_PATH = ROOT_PATH + 'datasets/' + DATASET_NAME + '/'
MODEL_PATH   = ROOT_PATH + 'models/'   + DATASET_NAME + '/'

In [0]:
def get_generator(directory,
                  image_shape=(224, 224),
                  batch_size=32,
                  should_augment=False,
                  data_gen_args=None):
    
    # only rescale
    if should_augment is False:
        image_gen = ImageDataGenerator(rescale=(1./255))
    
    # use dictionary to define augmentations
    else:
        if data_gen_args is None:
            data_gen_args = dict(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
        
        image_gen = ImageDataGenerator(**data_gen_args)
    
    return image_gen.flow_from_directory(directory,
                                         target_size=image_shape,
                                         batch_size=batch_size)

In [19]:
HEIGHT, WIDTH, CHANNELS = 224, 224, 3
BATCH_SIZE = 32

train_generator = get_generator(DATASET_PATH + 'train',
                                batch_size=BATCH_SIZE,
                                should_augment=DATA_AUGMENTATION)

test_generator = get_generator(DATASET_PATH + 'test',
                               batch_size=BATCH_SIZE)

Found 120 images belonging to 3 classes.
Found 120 images belonging to 3 classes.


In [0]:
def build_model(image_shape,
                classes,
                num_layers_to_freeze=21,
                base_model_weights_path=None,
                top_model_weights_path=None):
    
    model_name = f'vgg19_{num_layers_to_freeze}'
    
    # build the base model
    if base_model_weights_path is None:
        base_model_weights_path = 'imagenet'
    base_model = applications.vgg19.VGG19(input_shape=image_shape,
                                          weights=base_model_weights_path,
                                          include_top=False)
    print('Load VGG19 as base model')
    
    # load base model weights
    #base_model.load_weights(base_model_weights_path)
    
    # adding classification block on top of base model
    x = Flatten(input_shape=image_shape, name='flatten')(base_model.output)
    x = Dense(512, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)
    
    # combine both
    model = Model(inputs=base_model.input, outputs=x, name=model_name)
    
    # load top model weights
    #model.load_weights(top_model_weights_path)
    
    # freeze layers
    for i, layer in enumerate(model.layers):
        if i <= num_layers_to_freeze:
            layer.trainable = False
        else:
            layer.trainable = True
            print(f'Layer {i} {layer.name} is trainable')

    # compile model
    loss_type = 'binary_' if classes == 2 else 'categorical_'
    loss_type += 'crossentropy'
    model.compile(loss=loss_type,
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])
    
    print('Model compiled')
    return model

In [21]:
BASE_MODEL_WEIGHTS_PATH = ROOT_PATH + 'models/vgg19_weights_no_top.h5'
NUM_CLASSES = len(train_generator.class_indices)

model = build_model(image_shape=(HEIGHT, WIDTH, CHANNELS), classes=NUM_CLASSES)
model.summary()

Load VGG19 as base model
Layer 22 flatten is trainable
Layer 23 fc1 is trainable
Layer 24 dropout1 is trainable
Layer 25 fc2 is trainable
Layer 26 dropout2 is trainable
Layer 27 predictions is trainable
Model compiled
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
________________________________________________________

In [0]:
# fine-tune the model
EPOCHS = 300
TRAIN_LEN = train_generator.n
TEST_LEN = test_generator.n
CLASS_WEIGHT = None #{0: 1.0, 1: 0.4}

SAVE_HISTORY_PATH = MODEL_PATH + model.name + '.history'
SAVE_MODEL_PATH   = MODEL_PATH + model.name + '.model'

import pickle

def train_model(save_model_path,
                save_history_path,
                save_history=True):
    
    early_stopping = EarlyStopping(patience=20,
                                   monitor='val_loss',
                                   restore_best_weights=True)

    checkpoint = ModelCheckpoint(save_model_path,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=False)

    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
    #                              patience=5, min_lr=0.001)

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=TRAIN_LEN //  BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=test_generator,
        validation_steps=TEST_LEN // BATCH_SIZE,
        verbose=1,
        callbacks=[early_stopping, checkpoint],
        class_weight=CLASS_WEIGHT)

    # save history
    if save_history:
        with open(save_history_path, 'wb') as file:
            pickle.dump(history.history, file)
        print(f'\n\nSaved history into {save_history_path}')

In [23]:
train_model(save_model_path=SAVE_MODEL_PATH,
            save_history_path=SAVE_HISTORY_PATH)

Epoch 1/300
3/3 [==============================] - 66s 22s/step - loss: 2.2572 - acc: 0.2708 - val_loss: 1.2515 - val_acc: 0.3438

Epoch 00001: val_loss improved from inf to 1.25148, saving model to drive/My Drive/medical_image_recognition/models/miniMIT_Etus/vgg19_21.model
Epoch 2/300
3/3 [==============================] - 15s 5s/step - loss: 1.7054 - acc: 0.2846 - val_loss: 1.1087 - val_acc: 0.3409

Epoch 00002: val_loss improved from 1.25148 to 1.10865, saving model to drive/My Drive/medical_image_recognition/models/miniMIT_Etus/vgg19_21.model
Epoch 3/300
3/3 [==============================] - 1s 380ms/step - loss: 1.3887 - acc: 0.3830 - val_loss: 1.1454 - val_acc: 0.3636

Epoch 00003: val_loss did not improve from 1.10865
Epoch 4/300
3/3 [==============================] - 1s 372ms/step - loss: 1.5727 - acc: 0.3750 - val_loss: 1.2176 - val_acc: 0.3523

Epoch 00004: val_loss did not improve from 1.10865
Epoch 5/300
3/3 [==============================] - 1s 390ms/step - loss: 1.3306 -

In [0]:
def test_model(model, model_name, test_data, test_labels, batch_size=16):
    """
    TODO
    """
    test_loss, test_score = model.evaluate(test_data,
                                           test_labels,
                                           batch_size=16)

    print('Results for {} model.'.format(model_name))

    print('Loss : {}'.format(test_loss))
    print('Score : {}'.format(test_score))

    # predictions
    preds = model.predict(test_data, batch_size=16)
    preds = np.argmax(preds, axis=-1)

    # original labels
    orig_test_labels = np.argmax(test_labels, axis=-1)

    # shapes
    print(orig_test_labels.shape)
    print(preds.shape)

    ### confusion matrix
    cm  = confusion_matrix(orig_test_labels, preds)
    plt.figure()
    plot_confusion_matrix(cm,figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
    plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
    plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
    plt.show()

    ### metrics
    tn, fp, fn, tp = cm.ravel()
    precision = tp / (tp + fp)
    recall =    tp / (tp + fn)
    accuracy = (tp + tn) / (tn + fp + fn + tp)

    print("Recall of the model is {:.5f}".format(recall))
    print("Precision of the model is {:.5f}".format(precision))
    print("Accuracy of the model is {:.5f}".format(accuracy))

    return

In [0]:
from keras.models import load_model
# choose the weights to load into model
BEST_MODEL = MODEL_PATH + 'vgg19_21.model'
model = load_model(BEST_MODEL)

# let's test our model !
test_model(model, 'VGG19', data['test'], labels['test'], batch_size=16)

NameError: ignored